# Pipelines

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

In [5]:
dataset = pd.read_csv('cancer.csv')
dataset[:5]

,out,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
dataset.shape

(569, 31)

In [7]:
array = dataset.values
X = array[:,1:31]
y = array[:,0]

In [8]:
X = X.astype(np.float64)

In [9]:
y[-5:]

array(['M', 'M', 'M', 'M', 'B'], dtype=object)

In [10]:
# convert to 0 and 1

In [11]:
y[y=='M'] = 0
y[y=='B'] = 1

In [12]:
y[-5:]

array([0, 0, 0, 0, 1], dtype=object)

In [13]:
y = y.astype(np.float64)
y[-5:]

array([0., 0., 0., 0., 1.])

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

In [15]:
# scale training set in (0,1)

In [16]:
scaler = MinMaxScaler().fit(X_train)

In [17]:
# it is important to build the scaler with the train set only

In [18]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
svm = SVC(kernel='rbf',gamma=1)
svm.fit(X_train_scaled,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [20]:
# test set performance

In [21]:
svm.score(X_test_scaled,y_test)

0.972027972027972

## Pipeline

In [22]:
from sklearn.pipeline import Pipeline

In [23]:
pipe = Pipeline([('scaler',MinMaxScaler()), ('svm',SVC(kernel='rbf',gamma=1))])

In [25]:
pipe.fit(X_train,y_train)
pipe.score(X_test,y_test)

0.972027972027972

## Pipeline with cross_val_score

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [27]:
pipe = Pipeline([('scaler',MinMaxScaler()), ('svm',SVC(kernel='rbf',gamma=1))])
kfold = KFold(n_splits=10,random_state=1)
scores = cross_val_score(pipe,X,y,cv=kfold)
scores.mean()

0.9771303258145363

In [28]:
# at each split, only the train set is scaled with MinMaxScaler

## Pipeline with GridSearchCV

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
param_grid = {'svm__C':[0.001,0.01,0.1,1,10,100],
             'svm__gamma':[0.001,0.01,0.1,1,10,100]}

In [32]:
pipe = Pipeline([('scaler',MinMaxScaler()), ('svm',SVC(kernel='rbf'))])
grid = GridSearchCV(pipe,param_grid=param_grid,cv=10,iid=False)
grid.fit(X_train,y_train);

In [33]:
grid.best_params_

{'svm__C': 1, 'svm__gamma': 1}

In [34]:
param_grid = {'svm__C':[0.6,0.7,0.8,0.9,1,1.1,1.2,1.3],
             'svm__gamma':[0.6,0.7,0.8,0.9,1,1.1,1.2,1.3]}

In [35]:
pipe = Pipeline([('scaler',MinMaxScaler()), ('svm',SVC(kernel='rbf'))])
grid = GridSearchCV(pipe,param_grid=param_grid,cv=10,iid=False)
grid.fit(X_train,y_train);

In [36]:
grid.best_params_

{'svm__C': 1.2, 'svm__gamma': 1.1}

In [37]:
grid.score(X_test,y_test)

0.972027972027972

## make_pipeline

In [39]:
from sklearn.pipeline import make_pipeline

In [41]:
pipe = Pipeline([('scaler',MinMaxScaler()), ('svm',SVC(kernel='rbf',gamma=1))])

In [42]:
pipe2 = make_pipeline(MinMaxScaler(),SVC(kernel='rbf',gamma=1))